In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 25667933
paper_name = 'nislow_hammond_2015' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [6]:
files = ['TableS5_vsT1-hom1-dropsToBg.xlsx',
         'TableS6_vsT1-hom1-NaCl-dropsToBg.xlsx',
         'TableS8_vsT1-het1-dropsToBg.xlsx',
         'TableS9_vsT1-het1-NaCl-dropsToBg.xlsx']
sheet_names = ['ground','flight']

In [8]:
original_data_list = []
for f in files:
    for s in sheet_names:
        original_data = pd.read_excel('raw_data/' + f, sheet_name=s)
        
        original_data['orf'] = original_data['Strain'].apply(lambda x: x.split(':')[0])
        original_data['orf'] = clean_orf(original_data['orf'])
        original_data['orf'] = translate_sc(original_data['orf'], to='orf')
        # Make sure everything translated ok
        t = looks_like_orf(original_data['orf'])
        print(original_data.loc[~t,])
        original_data.set_index('orf', inplace=True)
        
        data_cols = [x for x in original_data.columns.values if 'Log2ratio' in x]
        original_data = original_data.loc[:,data_cols].copy()
        original_data = original_data.groupby(original_data.index).mean()
        
        print(original_data.shape)
        original_data_list.append(original_data)

Empty DataFrame
Columns: [Strain, ORF, Gene, Log2ratio, 14G, Log2ratio, 21G, P, 14G, P, 21G, Drops out, 14G, Drops out, 21G, Significant, 14G, Significant, 21G, orf]
Index: []
(4067, 2)
Empty DataFrame
Columns: [Strain, ORF, Gene, Log2ratio, 14G, Log2ratio, 21G, P, 14G, P, 21G, Drops out, 14G, Drops out, 21G, Significant, 14G, Significant, 21G, orf]
Index: []
(4002, 2)
Empty DataFrame
Columns: [Strain, ORF, Gene, Log2ratio, 14G, Log2ratio, 21G, P, 14G, P, 21G, Drops out, 14G, Drops out, 21G, Significant, 14G, Significant, 21G, orf]
Index: []
(4017, 2)
Empty DataFrame
Columns: [Strain, ORF, Gene, Log2ratio, 14G, Log2ratio, 21G, P, 14G, P, 21G, Drops out, 14G, Drops out, 21G, Significant, 14G, Significant, 21G, orf]
Index: []
(3844, 2)
Empty DataFrame
Columns: [Strain, ORF, Gene, Log2ratio, 21G, P, 21G, Drops out, 21G, Significant, 21G, orf]
Index: []
(2807, 1)
Empty DataFrame
Columns: [Strain, ORF, Gene, Log2ratio, 21G, P, 21G, Drops out, 21G, Significant, 21G, orf]
Index: []
(5153, 1)


In [32]:
original_data = pd.concat(original_data_list, axis=1)

In [33]:
original_data.index.name='orf'

In [34]:
# Data represent log(gen1/gen2) such that, originally, higher values correspond to more extreme depletion
original_data = -original_data

In [35]:
original_data.head()

,"Log2ratio, 14G","Log2ratio, 21G","Log2ratio, 14G","Log2ratio, 21G","Log2ratio, 14G","Log2ratio, 21G","Log2ratio, 14G","Log2ratio, 21G","Log2ratio, 21G","Log2ratio, 21G","Log2ratio, 14G","Log2ratio, 21G","Log2ratio, 14G","Log2ratio, 21G"
orf,,,,,,,,,,,,,,
YAL002W,-0.742743,-3.173170,0.928157,0.414255,-1.085242,1.014800,-1.471382,-0.967316,NaN,NaN,NaN,NaN,NaN,NaN
YAL004W,-0.636380,-0.192795,1.188308,0.739977,-0.922251,-2.007862,-1.210132,-0.554925,NaN,NaN,NaN,NaN,0.419885,-0.172162
YAL005C,0.324102,0.761777,1.509200,1.054024,-0.010577,-0.608062,-0.078935,0.287202,NaN,0.136482,NaN,NaN,NaN,NaN
YAL007C,0.082658,-0.264514,-0.146598,-0.135115,-0.016527,0.902277,0.186224,-0.010792,NaN,-0.455655,NaN,NaN,-0.435725,-0.912295
YAL008W,-0.121101,-0.559293,0.587159,0.522789,-0.648085,0.209181,-0.691597,-0.075703,NaN,-0.601332,NaN,NaN,0.767809,-0.502311


# Prepare the final dataset

In [36]:
data = original_data.copy()

In [37]:
dataset_ids = [5276, 5277, 5278, 5279, 5280, 5281, 5282, 5283, 5284, 5285, 5286, 5287, 5288, 5289]
datasets = datasets.reindex(index=dataset_ids)

In [38]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [39]:
data.head()

dataset_id,5276,5277,5278,5279,5280,5281,5282,5283,5284,5285,5286,5287,5288,5289
data_type,value,value,value,value,value,value,value,value,value,value,value,value,value,value
orf,,,,,,,,,,,,,,
YAL002W,-0.742743,-3.173170,0.928157,0.414255,-1.085242,1.014800,-1.471382,-0.967316,NaN,NaN,NaN,NaN,NaN,NaN
YAL004W,-0.636380,-0.192795,1.188308,0.739977,-0.922251,-2.007862,-1.210132,-0.554925,NaN,NaN,NaN,NaN,0.419885,-0.172162
YAL005C,0.324102,0.761777,1.509200,1.054024,-0.010577,-0.608062,-0.078935,0.287202,NaN,0.136482,NaN,NaN,NaN,NaN
YAL007C,0.082658,-0.264514,-0.146598,-0.135115,-0.016527,0.902277,0.186224,-0.010792,NaN,-0.455655,NaN,NaN,-0.435725,-0.912295
YAL008W,-0.121101,-0.559293,0.587159,0.522789,-0.648085,0.209181,-0.691597,-0.075703,NaN,-0.601332,NaN,NaN,0.767809,-0.502311


## Subset to the genes currently in SGD

In [40]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 24


In [41]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,5276,5277,5278,5279,5280,5281,5282,5283,5284,5285,5286,5287,5288,5289
,data_type,value,value,value,value,value,value,value,value,value,value,value,value,value,value
gene_id,orf,,,,,,,,,,,,,,
2,YAL002W,-0.742743,-3.173170,0.928157,0.414255,-1.085242,1.014800,-1.471382,-0.967316,NaN,NaN,NaN,NaN,NaN,NaN
1863,YAL004W,-0.636380,-0.192795,1.188308,0.739977,-0.922251,-2.007862,-1.210132,-0.554925,NaN,NaN,NaN,NaN,0.419885,-0.172162
4,YAL005C,0.324102,0.761777,1.509200,1.054024,-0.010577,-0.608062,-0.078935,0.287202,NaN,0.136482,NaN,NaN,NaN,NaN
5,YAL007C,0.082658,-0.264514,-0.146598,-0.135115,-0.016527,0.902277,0.186224,-0.010792,NaN,-0.455655,NaN,NaN,-0.435725,-0.912295
6,YAL008W,-0.121101,-0.559293,0.587159,0.522789,-0.648085,0.209181,-0.691597,-0.075703,NaN,-0.601332,NaN,NaN,0.767809,-0.502311


# Normalize

In [42]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [43]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [44]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,5276,5277,5278,5279,5280,5281,5282,5283,5284,5285,...,5280,5281,5282,5283,5284,5285,5286,5287,5288,5289
,data_type,value,value,value,value,value,value,value,value,value,value,...,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,,,,,,,,,,
2,YAL002W,-0.742743,-3.173170,0.928157,0.414255,-1.085242,1.014800,-1.471382,-0.967316,NaN,NaN,...,-1.447273,0.854387,-2.487992,-1.704310,NaN,NaN,NaN,NaN,NaN,NaN
1863,YAL004W,-0.636380,-0.192795,1.188308,0.739977,-0.922251,-2.007862,-1.210132,-0.554925,NaN,NaN,...,-1.211421,-2.280717,-2.043433,-0.900673,NaN,NaN,NaN,NaN,0.623349,-0.260107
4,YAL005C,0.324102,0.761777,1.509200,1.054024,-0.010577,-0.608062,-0.078935,0.287202,NaN,0.136482,...,0.107801,-0.828845,-0.118517,0.740399,NaN,0.213810,NaN,NaN,NaN,NaN
5,YAL007C,0.082658,-0.264514,-0.146598,-0.135115,-0.016527,0.902277,0.186224,-0.010792,NaN,-0.455655,...,0.099191,0.737679,0.332694,0.159691,NaN,-0.898798,NaN,NaN,-0.953532,-1.518379
6,YAL008W,-0.121101,-0.559293,0.587159,0.522789,-0.648085,0.209181,-0.691597,-0.075703,NaN,-0.601332,...,-0.814694,0.018800,-1.161061,0.033197,NaN,-1.172522,NaN,NaN,1.264568,-0.821380


# Print out

In [45]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [46]:
from IO.save_data_to_db3 import *

In [47]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/14 [00:00<?, ?it/s]

Deleting all datasets for PMID 25667933...
Inserting the new data...


100%|██████████| 14/14 [01:55<00:00,  8.23s/it]

Updating the data_modified_on field...
